In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
df_sp = pd.read_csv(r'F:\Abhilah_First\Results\Era5Land\noised_stake_data.csv')
df_sp.head()

,Unnamed: 0,Julian Day,obs_period(days),elevation(m),Derbis thickness (cm),melt (cm),pdd,Model-1 Melt Rate (cm/day),Model-2 Melt Rate (cm/day),Model-3 Melt Rate (cm/day)
0,0,232,17,3868.0,116.704707,10.789589,178.524402,0.960872,0.522011,0.543095
1,1,250,18,3867.0,113.487625,5.006354,152.118696,0.773263,0.420089,0.437056
2,2,263,13,3867.0,114.203133,4.979968,100.791271,0.709409,0.385399,0.400965
3,3,273,10,3868.0,116.584423,10.297328,72.470200,0.663096,0.360239,0.374789
4,4,298,25,3868.0,109.178897,0.539409,130.864899,0.478961,0.260204,0.270714


In [2]:
df_model1_param = pd.read_csv(r'F:\Abhilah_First\Results\Era5Land\Parameters_1000.csv')
df_model1_param.head(4)

,Unnamed: 0,TF1 Model 1,TF2 Model 1,MSE Model 1,DDF Model 2,d0 Model 2,MSE Model 2,DDF Model 3,d0 Model 3,a Model 3,MSE Model 3
0,0,1.098285,-0.328110,0.505590,4.286001,17.206201,0.437698,3.702612,11.623785,0.444232,0.427410
1,1,1.196568,-0.287907,0.504975,5.024441,12.488065,0.393736,4.370584,6.361340,0.350399,0.396229
2,2,1.111208,-0.328610,0.533966,4.615500,14.941232,0.454123,3.942050,9.816933,0.422546,0.450480
3,3,1.043782,-0.375285,0.481698,4.650498,15.363376,0.426325,3.825529,11.479997,0.446073,0.422991


In [3]:
data = {'DDF':df_model1_param['DDF Model 2']} 
df_all_iterations = pd.DataFrame(data)
df_all_iterations['d0'] = df_model1_param['d0 Model 2']
MSEs = list(df_model1_param['MSE Model 2'])
df_all_iterations['MSE'] = df_model1_param['MSE Model 2']
weight = []
for i in MSEs:
    weight.append(math.exp(-i))
df_all_iterations['Weight'] = weight
df_all_iterations.head(3)

,DDF,d0,MSE,Weight
0,4.286001,17.206201,0.437698,0.645521
1,5.024441,12.488065,0.393736,0.674532
2,4.615500,14.941232,0.454123,0.635005


In [4]:
df_all_iterations['DDF*Weight'] = df_all_iterations['DDF']*df_all_iterations['Weight']
df_all_iterations['d0*Weight'] = df_all_iterations['d0']*df_all_iterations['Weight']

df_all_iterations['Weight*(DDF - xBARw)^2'] = df_all_iterations['Weight']*(df_all_iterations['DDF'] - sum(df_all_iterations['DDF*Weight']/1000) )**2
df_all_iterations['Weight*(d0 - yBARw)^2'] = df_all_iterations['Weight']*(df_all_iterations['d0'] - sum(df_all_iterations['d0*Weight']/1000) )**2

df_all_iterations.head()

,DDF,d0,MSE,Weight,DDF*Weight,d0*Weight,Weight*(DDF - xBARw)^2,Weight*(d0 - yBARw)^2
0,4.286001,17.206201,0.437698,0.645521,2.766703,11.106959,0.875066,37.461457
1,5.024441,12.488065,0.393736,0.674532,3.389148,8.423604,2.442092,5.672032
2,4.615500,14.941232,0.454123,0.635005,2.930865,9.487754,1.416974,18.195582
3,4.650498,15.363376,0.426325,0.652904,3.036328,10.030807,1.525981,21.775578
4,4.659066,14.514407,0.408834,0.664425,3.095598,9.643728,1.570363,16.123499


In [5]:
ddf_variability = (sum(df_all_iterations['Weight*(DDF - xBARw)^2'])*1000/(1000*(1000-1)*sum(df_all_iterations['Weight'])) )**(0.5)


d0_variability = (sum(df_all_iterations['Weight*(d0 - yBARw)^2'])*1000/(1000*(1000-1)*sum(df_all_iterations['Weight'])) )**(0.5)

print(df_all_iterations['DDF'][MSEs.index(min(MSEs))], df_all_iterations['d0'][MSEs.index(min(MSEs))])
print(round(ddf_variability,2), round(d0_variability,2))
print('RMSE =',min(MSEs)**0.5)

5.659874186708133 9.77983904893038
0.05 0.16
RMSE = 0.5791293092059638
